# Surrounding Landscape

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

method = 'mapbiomas'
biomass_raw = ee.Image(f"{data_folder}/ESA_CCI_2020")

## Surrounding mature forest biomass

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1) at a 500m distance

In [30]:
if method == 'tmf':
    transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
    mature_mask = transition.eq(10).selfMask()
    age = ee.Image(f"{data_folder}/tmf_{last_year}")

elif method == 'mapbiomas':
    lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
                .select([f"classification_{year}" for year in config.range_1985_2020])
                .byte()
                .rename([str(year) for year in config.range_1985_2020]))
    mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()
    age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

mature_biomass = biomass_raw.updateMask(mature_mask).rename("mature_biomass")

# keep only forests that are 1500m away from any non-forest area
edge_detec = mature_mask.unmask(-1).zeroCrossing()
distance_to_forest_edge = edge_detec.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("dist")
distance_gt_1500 = distance_to_forest_edge.gt(1500).selfMask().updateMask(mature_mask)

mature_biomass = mature_biomass.rename("mature_biomass")
mature_biomass_exclude_edge = biomass_raw.updateMask(distance_gt_1500).rename("mature_biomass")

sur_cover = mature_mask.unmask(0).focalMean(radius = 500, units = "meters").float().updateMask(age).rename("sur_cover")

Export at a 100m resolution (same as ESA CCI Biomass) to argue for the distance-to-edge effect.

In [31]:
export_image(distance_to_forest_edge, "distance_to_forest_edge", region = roi, scale = 30)
export_image(sur_cover, "sur_cover_500m", region = roi, scale = 30)
export_image(mature_biomass, "mature_biomass", region = roi, scale = 100)
export_image(mature_biomass_exclude_edge, "mature_biomass_exclude_edge_1500m", region = roi, scale = 500)

## Biomass of nearest mature forest

In [32]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_exclude_edge_1500m")

mature_biomass_10km = mature_biomass.reproject(
    crs = mature_biomass.projection(),  # Use the same projection as the original image
    scale = 10000  # Set the resolution to 10 km (10000 meters)
)

nearest_mature_biomass = mature_biomass_10km \
    .reduceNeighborhood(
        reducer=ee.Reducer.first(),
        kernel=ee.Kernel.euclidean(250, 'pixels'),
        skipMasked=False
    ).rename("nearest_mat_biomass")

export_image(nearest_mature_biomass, "nearest_mature_biomass_10k", region = roi, scale = 10000)

# map = geemap.Map()
# # map.addLayer(mature_biomass_neighborhood, {'min': 1, 'max': 400, 'palette': ['yellow', 'blue']}, "mature_biomass_neighborhood")
# map.addLayer(mature_biomass, {'min': 1, 'max': 400, 'palette': ['yellow', 'blue']}, "mature_biomass", opacity = 0.5)
# # map.addLayer(mature_biomass_10km, {'min': 1, 'max': 400, 'palette': ['yellow', 'blue']}, "mature_biomass_10km")
# map.addLayer(nearest_mature_biomass, {'min': 1, 'max': 400, 'palette': ['yellow', 'blue']}, "nearest_mature_biomass")
# map

## Amazon Quarters
Mean Biomass Value Per Climatic Region

In [7]:
# Load the mature biomass image
mature_biomass_exclude_edge = ee.Image(f"{data_folder}/mapbiomas/mature_biomass_exclude_edge_1500m")

# Load the individual feature collections and combine them into a single FeatureCollection
quarters = ee.FeatureCollection([
    ee.FeatureCollection(f"{data_folder}/raw/NE_outline_no_overlap").map(lambda f: f.set('sector', 1)),
    ee.FeatureCollection(f"{data_folder}/raw/NW_outline_no_overlap").map(lambda f: f.set('sector', 2)),
    ee.FeatureCollection(f"{data_folder}/raw/SW_outline_no_overlap").map(lambda f: f.set('sector', 3)),
    ee.FeatureCollection(f"{data_folder}/raw/SE_outline_no_overlap").map(lambda f: f.set('sector', 4)),
]).flatten()

ecoregions = (
    ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    .filterBounds(roi)
    .map(lambda feature: feature.intersection(roi))
)

# Function to get mean biomass per Feature
def create_biomass_image(feature):
    mean_biomass = mature_biomass.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 500,
        maxPixels = 1e12
    )
    return feature.set(mean_biomass)

# Apply the function to each feature in the FeatureCollection
mean_biomass_quarters = quarters.map(create_biomass_image)
mean_biomass_ecoregions = ecoregions.map(create_biomass_image)

# Helper function to create an image from a property in the feature collection
def create_property_image(feature_collection, property_name):
    return feature_collection.reduceToImage(
        properties = [property_name],
        reducer = ee.Reducer.first()
    ).rename(property_name)

# Create images for the 'mature_biomass' and 'sector' properties
quarter_biomass = create_property_image(mean_biomass_quarters, 'mature_biomass')
quarter = create_property_image(mean_biomass_quarters, 'sector')
ecoreg_biomass = create_property_image(mean_biomass_quarters, 'mature_biomass')
ecoreg = create_property_image(mean_biomass_quarters, 'ECO_ID')

quarters_ecoreg_biomass = ee.Image.cat([quarter_biomass, quarter, ecoreg_biomass, ecoreg]).rename(["quarter_biomass", "quarter", "ecoreg_biomass", "ecoreg"])

In [8]:
export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass", region = roi, scale = 5000)